In [3]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
# %pip install osmnet
import pandana
# print(pandana.__version__)

pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import warnings
warnings.filterwarnings("ignore")

/home/rongxiang/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [72]:
%%time
df_order = pd.read_csv("./data/Dubai_all_delivery_202310.csv", sep=",", header=0)
# print(df_order.columns)
print('total number of orders including both food and grocery in this dataset:',df_order.shape[0])
# df_order = df_order[df_order['main_cuisine'].notna()]

total number of orders including both food and grocery in this dataset: 3617110
CPU times: user 10.3 s, sys: 10.1 s, total: 20.4 s
Wall time: 1min 4s


In [5]:
df_order = df_order[~df_order.vendor_id.isin([677914,690257])] # these vendor's location is problematic so exclude it 
df_order =  df_order[df_order.vertical=='food'] # select food order only; Grocery orders are not so reliable because Talabat mostly only deliver groceries from their own warehouses
print('total food orders:',df_order.shape[0])

In [76]:
df_order.head(2)

,account_id,order_id,order_date,order_time,vendor_id,vendor_name,main_cuisine,vertical,delivery_fee_amount_lc,service_fee_amount_lc,basket_amount_lc,is_incentive_applied,incentive_amount_lc,pickup_id,customer_lon,customer_lat,vendor_lon,vendor_lat,order_time_hour,delivery_duration,total_duration
0,4285647,1305786978,2023-10-10,2023-10-10 18:37:22,15921,"312 , Al Sufouh 1",american,food,9.00000,2.36000,78.50000,NaN,NaN,249215302.00000,55.22939,25.16458,55.17584,25.10785,18,37,54
1,6623723,1304803532,2023-10-10,2023-10-10 01:59:24,680435,Bao,asian,food,4.00000,2.82000,94.00000,True,20.00000,249057659.00000,55.51919,25.23093,55.50995,25.22327,1,5,20


In [6]:
df_vendor = pd.read_csv('./data/all_vendor_oct.csv',header=0)
df_vendor = df_vendor[df_vendor.vertical=='food']
print(df_vendor.shape[0]) #8106
df_vendor.head()  

8106


,vertical,vendor_id,main_cuisine,vendor_name,vendor_lon,vendor_lat
0,food,682312,coffee,"Starbucks, Al Jada",55.48008,25.31986
1,food,663746,emirati,"Harees & Mahashi, Al Mujarah",55.39292,25.36944
2,food,39238,yemeni,"Al Rukn Al Arabi, Al Sawan,Al Rashidiya 3",55.43238,25.39337
3,food,714271,seafood,"Not Only Fish, Jumeirah Lakes Towers - JLT",55.14158,25.06848
4,food,709554,pizza,"Pasta La Pizza, Bluewaters Island",55.12460,25.07763


In [7]:
food_categories = {
    'mexican': 'mexican',
    'coffee': 'coffee',
    'burgers': 'fast food',
    'juices': 'coffee',
    'russian': 'russian',
    'indian': 'indian',
    'breakfast': 'breakfast',
    'shawarma': 'arabic',
    'cafeteria': 'coffee',
    'pizza': 'fast food',
    'african': 'african',
    'arabic sweets': 'dessert',
    'sandwiches': 'fast food',
    'arabic': 'arabic',
    'chinese': 'chinese',
    'desserts': 'dessert',
    'egyptian': 'egyptian',
    'fried chicken': 'fast food',
    'lebanese': 'lebanese',
    'chicken': 'chicken',
    'donuts': 'dessert',
    'tunisian': 'tunisian',
    'american': 'american',
    'japanese': 'japanese',
    'south indian': 'indian',
    'sushi': 'japanese',
    'international': 'international',
    'ice cream': 'dessert',
    'iranian': 'iranian',
    'koshary': 'egyptian',
    'cafe': 'coffee',
    'grills': 'bbq',
    'healthy': 'healthy',
    'afghani': 'afghani',
    'asian': 'asian',
    'mediterranean': 'mediterranean',
    'keto': 'keto',
    'cakes and desserts': 'dessert',
    'north indian': 'indian',
    'pakistani': 'pakistani',
    'filipino': 'filipino',
    'palestinian': 'palestinian',
    'fast food': 'fast food',
    'wings': 'fast food',
    'mandi': 'yemeni',
    'emirati': 'emirati',
    'italian': 'italian',
    'bakery': 'breakfast',
    'moroccan': 'moroccan',
    'uzbek': 'uzbek',
    'turkish': 'turkish',
    'pastries': 'dessert',
    'bubble tea': 'dessert',
    'salad': 'salad',
    'pastries & sweets': 'dessert',
    'acai': 'acai',
    'kebab': 'bbq',
    'soup': 'soup',
    'seafood': 'seafood',
    'vietnamese': 'vietnamese',
    'pasta': 'italian',
    'nepalese': 'nepalese',
    'street food': 'fast food',
    'beverages': 'dessert',
    'yemeni': 'yemeni',
    'snacks': 'dessert',
    'european': 'european',
    'doner': 'turkish',
    'frozen yogurt': 'dessert',
    'jordanian': 'jordanian',
    'cakes': 'dessert',
    'biryani': 'indian',
    'poke': 'seafood',
    'indonesian': 'indonesian',
    'thai': 'thai',
    'falafel': 'arabic',
    'manaqeesh': 'arabic',
    'korean': 'korean',
    'milkshakes': 'dessert',
    'steaks': 'bbq',
    'dumplings': 'chinese',
    'wraps': 'fast food',
    'tea': 'coffee',
    'vegan': 'vegetarian',
    'hot dogs': 'fast food',
    'crepes': 'dessert',
    'curry': 'indian',
    'middle eastern': 'middle eastern',
    'greek': 'greek',
    'waffles': 'breakfast',
    'vegetarian': 'vegetarian',
    'bowls': 'bowls',
    'saudi': 'saudi',
    'dim sum': 'chinese',
    'noodles': 'chinese',
    'mongolian': 'mongolian',
    'brazilian': 'brazilian',
    'singaporean': 'singaporean',
    'burrito': 'mexican',
    'bakery and cakes': 'dessert',
    'nan': None,
    'bagel': 'breakfast',
    'british': 'british',
    'gluten free': 'gluten free',
    'tacos': 'mexican',
    'syrian': 'syrian',
    'iraqi': 'iraqi',
    'bread & bakery': 'breakfast',
    'sliders': 'sliders',
    'portuguese': 'portuguese',
    'georgian': 'georgian',
    'bbq': 'bbq',
    'mezze': 'arabic',
    'smoothies': 'dessert',
    'gelato': 'dessert',
    'fish & chips': 'seafood',
    'kuwaiti': 'kuwaiti',
    'calzone': 'italian',
    'churros': 'dessert',
    'french': 'french',
    'argentinian': 'argentinian',
    'latin american': 'latin american',
    'specialty_store': 'specialty store',
    'jamaican': 'jamaican',
    'peruvian': 'peruvian',
    'spanish': 'spanish',
    'pancakes': 'breakfast'
}

# merge cuisines

In [8]:
%time
df_vendor['main_cuisine'] = df_vendor['main_cuisine'].replace(food_categories)
df_order['main_cuisine'] = df_order['main_cuisine'].replace(food_categories)
df_vendor = df_vendor[~df_vendor['main_cuisine'].isna()]
df_order = df_order[~df_order['main_cuisine'].isna()]
print('unique types of cuisine:',df_vendor.main_cuisine.unique().shape[0])
print('unique types of cuisine:',df_order.main_cuisine.unique().shape[0])

CPU times: user 6 µs, sys: 4 µs, total: 10 µs
Wall time: 18.6 µs


In [83]:
df_vendor.to_csv('./data/all_vendor_oct_processed.csv',index=False)

In [84]:
df_order.to_csv('./data/Dubai_all_delivery_202310_processed.csv',index=False)

In [85]:
# %%time
# from shapely.geometry import Point
# from geopandas import GeoDataFrame
# import geopandas as gpd
# geometry = [Point(xy) for xy in zip(df_vendor.vendor_lon, df_vendor.vendor_lat)]
# gdf_vendor = GeoDataFrame(df_vendor, crs="EPSG:4326", geometry=geometry)
# gdf_vendor = gdf_vendor.rename(columns={"vendor_id": "n_vendor_id","main_cuisine":"n_main_cuisine",'vendor_name':'n_vendor_name',
#                                        'vendor_lon':'n_vendor_lon','vendor_lat':'n_vendor_lat','vertical':'n_vertical'})
# gdf_vendor = gdf_vendor.to_crs(epsg=3857)

In [142]:
# gdf_vendor.explore()